In [14]:
import os
import django
import pandas as pd
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fxfutbol.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [15]:
import ssl
import http.client
import json
import pandas as pd

class SofaScoreAPI:
    def __init__(self, match_id):
        self.match_id = match_id
        self.base_url = 'www.sofascore.com'

    def _make_request(self, endpoint):
        """
        Belirtilen endpoint'e HTTP isteği yapar ve yanıtı döner.
        """
        context = ssl._create_unverified_context()
        con = http.client.HTTPSConnection(self.base_url, context=context)
        try:
            con.request("GET", endpoint)
            response = con.getresponse()
            data = response.read()
            data = json.loads(data)
            return {"match_id":self.match_id,"response_code": response.status, "data": data}
        except Exception as e:
            con.request("GET", endpoint)
            response = con.getresponse()
            return {"match_id":self.match_id,"response_code": response.status , "data": None }

    def get_lineups(self):
        """
        Kadro bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/lineups'
        return self._make_request(endpoint)

    def get_incidents(self):
        """
        Maç olaylarını döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/incidents'
        return self._make_request(endpoint)

    def get_graph(self):
        """
        Maç baskı grafiğini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/graph'
        return self._make_request(endpoint)

    def get_match_info(self):
        """
        Maç bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}'
        return self._make_request(endpoint)

    def get_odds(self):
        """
        Maç oran bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/odds/672/featured'
        return self._make_request(endpoint)

    def get_pregame_form(self):
        """
        Takım form bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/pregame-form'
        return self._make_request(endpoint)

    def get_statistics(self):
        """
        Maç istatistiklerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/statistics'
        return self._make_request(endpoint)

    def fetch_all_data(self):
        """
        Tüm API endpoint'lerinden verileri toplar ve bir dict olarak döner.
        """
        return {
            "match_id": self.match_id,
            "lineups": self.get_lineups(),
            "incidents": self.get_incidents(),
            "graph": self.get_graph(),
            "match_info": self.get_match_info(),
            "odds": self.get_odds(),
            "pregame_form": self.get_pregame_form(),
            "statistics": self.get_statistics(),
        }
        
match_id = 12643086
sofa_api = SofaScoreAPI(match_id)
result_all = sofa_api.get_lineups()
result_all

{'match_id': 12643086,
 'response_code': 200,
 'data': {'confirmed': True,
  'home': {'players': [{'player': {'name': 'Gregory Obeng Sekyere',
      'firstName': '',
      'lastName': '',
      'slug': 'gregory-obeng-sekyere',
      'shortName': 'G. O. Sekyere',
      'position': 'G',
      'jerseyNumber': '30',
      'height': 177,
      'userCount': 42,
      'id': 1218039,
      'country': {'alpha2': 'GH',
       'alpha3': 'GHA',
       'name': 'Ghana',
       'slug': 'ghana'},
      'marketValueCurrency': 'EUR',
      'dateOfBirthTimestamp': 1071014400},
     'shirtNumber': 30,
     'jerseyNumber': '30',
     'position': 'G',
     'substitute': False,
     'statistics': {'goals': 0, 'ownGoals': 0, 'minutesPlayed': 90}},
    {'player': {'name': 'Edward Agyemang',
      'slug': 'edward-agyemang',
      'shortName': 'E. Agyemang',
      'jerseyNumber': '13',
      'userCount': 0,
      'id': 1941604,
      'country': {},
      'marketValueCurrency': 'EUR'},
     'shirtNumber': 13,
   

In [18]:
import http.client
import pandas as pd
import json
from datetime import datetime, timedelta
from analysis.models import MatchStats  # `MatchStats` modelini içe aktarın


class ResultAPI:
    def __init__(self, date="2023-01-01"):
        self.date = date
        self.full_df = pd.DataFrame()

    def AddDataBase(self):

            df = self.gunluk_fikstur()
            print(df)
            
            MatchStats.objects.filter(id__in=df['id'].tolist()).delete()
            
            fixture_list = [
                MatchStats(
                    id=row['id'],  # ID manuel olarak girilecek
                    tarih=row['tarih'],  # Tarih alanı ekleniyor
                    stats=row['Stats']  # JSON formatındaki veriyi 'stats' alanına ekle
                )
                for index, row in df.iterrows()
            ]
            
            MatchStats.objects.bulk_create(fixture_list)
            print("Mevcut veriler silindi ve yeni veriler eklendi.")
       
        
    def gunluk_fikstur(self):
        try:
            conn = http.client.HTTPSConnection('www.sofascore.com')
            conn.request(
                'GET',
                f'/api/v1/sport/football/scheduled-events/{self.date}'
            )
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)["events"]

            model_data = [{
                'tarih': self.date,
                'data': data,
              
            }]
            
            df = pd.DataFrame(model_data)
        except Exception as e:
            model_data = [{
                'tarih': self.date,
                'data': [],
                
            }]
        try:
            
            df = pd.DataFrame(model_data)
            df_exploded = df.explode('data')
            expanded_data = pd.json_normalize(df_exploded['data'])
            expanded_data['tarih'] = list(df_exploded['tarih'])
            data = expanded_data.drop_duplicates(subset='id', keep='last')
            data = data[data["status.type"] == "finished"]
            data = data[data["tournament.uniqueTournament.id"]==17]

            
        except:
            pass
        
   
        data_stat = []
        toplam = len(data)
        for idx in list(data["id"]):
            toplam = toplam-1
            
            try:
               
                sofa_api = SofaScoreAPI(idx)
                result_all = sofa_api.fetch_all_data()
                data_stat.append(result_all)
                print("{} işlem başarılı kalan işlem sayısı".format(toplam))
            except:
                print("{} işlem ERRORRRR kalan işlem sayısı".format(toplam))
                result_all = {}
                data_stat.append(result_all)
                
                
        data["Stats"] = data_stat

        columns = ['id','tarih','Stats']
        result_data = data[columns]

        return result_data
        
api = ResultAPI(date = "2024-10-06")
data  = api.AddDataBase()
data

8 işlem başarılı kalan işlem sayısı
7 işlem başarılı kalan işlem sayısı
6 işlem başarılı kalan işlem sayısı
5 işlem başarılı kalan işlem sayısı
4 işlem başarılı kalan işlem sayısı
3 işlem başarılı kalan işlem sayısı
2 işlem başarılı kalan işlem sayısı
1 işlem başarılı kalan işlem sayısı
0 işlem başarılı kalan işlem sayısı
         id       tarih                                              Stats
0  12437004  2024-10-06  {'match_id': 12437004, 'lineups': {'match_id':...
1  12437008  2024-10-06  {'match_id': 12437008, 'lineups': {'match_id':...
2  12437020  2024-10-06  {'match_id': 12437020, 'lineups': {'match_id':...
3  12437022  2024-10-06  {'match_id': 12437022, 'lineups': {'match_id':...
4  12437024  2024-10-06  {'match_id': 12437024, 'lineups': {'match_id':...
5  12437018  2024-10-06  {'match_id': 12437018, 'lineups': {'match_id':...
6  12437006  2024-10-06  {'match_id': 12437006, 'lineups': {'match_id':...
7  12437013  2024-10-06  {'match_id': 12437013, 'lineups': {'match_id':...
8

In [8]:
import ssl
import http.client
import json
import pandas as pd

def gunluk_fikstur(date):
        
            conn = http.client.HTTPSConnection('www.sofascore.com')
            conn.request(
                'GET',
                f'/api/v1/sport/football/scheduled-events/{date}'
            )
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)["events"]

            model_data = [{
                'tarih': date,
                'data': data,
              
            }]
            
            df = pd.DataFrame(model_data)
            return df
    
data = gunluk_fikstur("2024-10-06")
df_exploded = data.explode('data')
expanded_data = pd.json_normalize(df_exploded['data'])
expanded_data['tarih'] = list(df_exploded['tarih'])
data = expanded_data.drop_duplicates(subset='id', keep='last')
data = data[data["status.type"] == "finished"]
datam = data[data["tournament.uniqueTournament.id"]==17]
datam

,customId,winnerCode,hasGlobalHighlights,hasEventPlayerStatistics,hasEventPlayerHeatMap,detailId,crowdsourcingDataDisplayEnabled,id,startTimestamp,slug,...,tournament.uniqueTournament.primaryColorHex,tournament.uniqueTournament.secondaryColorHex,season.seasonCoverageInfo.editorCoverageLevel,roundInfo.name,roundInfo.cupRoundType,homeScore.overtime,awayScore.overtime,time.injuryTime3,time.injuryTime4,tarih
0,RV,1.0,True,True,True,1.0,False,12437004,1728136800,southampton-arsenal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
1,dsab,1.0,True,True,True,1.0,False,12437008,1728136800,brentford-wolverhampton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
2,Gskb,1.0,True,True,True,1.0,False,12437020,1728136800,bournemouth-leicester-city,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
3,rsT,1.0,True,True,True,1.0,False,12437022,1728136800,fulham-manchester-city,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
4,HsM,1.0,True,True,True,1.0,False,12437024,1728136800,west-ham-united-ipswich-town,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
5,OY,3.0,True,True,True,1.0,False,12437018,1728145800,everton-newcastle-united,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
6,KP,3.0,True,True,True,1.0,False,12437006,1728219600,aston-villa-manchester-united,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
7,osN,3.0,True,True,True,1.0,False,12437013,1728219600,chelsea-nottingham-forest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06
8,FsI,1.0,True,True,True,1.0,False,12437011,1728228600,tottenham-hotspur-brighton-and-hove-albion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-06


In [6]:
data.to_excel("Hamfiksture.xlsx")